In [8]:
# imports

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [9]:
# parameters
SEED = 2023
torch.manual_seed(SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
device

device(type='cuda')

In [42]:
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Input((21 * 2, )),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(20, activation='relu'),
#     tf.keras.layers.Dropout(0.4),
#     tf.keras.layers.Dense(10, activation='relu'),
#     tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
# ])

class TinyModel(torch.nn.Module):

    def __init__(self):
        super(TinyModel, self).__init__()

        self.linear1 = torch.nn.Linear(63, 100)
        self.linear2 = torch.nn.Linear(100, 50)
        self.linear3 = torch.nn.Linear(50, 5)

        self.Dropout = torch.nn.Dropout(p=0.2)
        self.activation = torch.nn.ReLU()
        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, x):
        x = self.linear1(x)
        x = self.Dropout(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.Dropout(x)
        x = self.activation(x)
        x = self.linear3(x)
        x = self.Dropout(x)
        x = self.activation(x)
        x = self.softmax(x)
        return x

net = TinyModel().to(device=device)

In [43]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

### Generating Dataset

In [13]:
import os
from os import listdir
from os.path import abspath, join, dirname
# dataset_path = join(abspath(path=''), 'Dataset')
# data = []
# for i, file in enumerate(listdir(dataset_path)):
#     d = np.load(join(dataset_path, file))
#     d = d.reshape(d.shape[0], -1)[1:]
#     d = d - d[0]
#     d = np.concatenate((d, np.full((d.shape[0], 1), i)), axis=1)
#     data.append(d)


In [14]:
from numpy import genfromtxt
data = genfromtxt(join(abspath(path=''), 'Dataset', 'keypoint.csv'), delimiter=',')

In [15]:
# data = np.concatenate(data)
data = torch.tensor(data, dtype=torch.float32)

In [ ]:
data.shape

torch.Size([4787, 43])

In [16]:
dataset = data.to(device=device)

In [17]:
dataset.shape

torch.Size([4787, 43])

In [18]:
from torch.utils.data import Dataset

class Dataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        data = self.dataset[index]
        return data[1:], data[0]
        # return self.dataframe.iloc[index]

In [19]:
dataset = Dataset(dataset)
split_ratio = [0.7, 0.3]
train_set, test_set = torch.utils.data.random_split(dataset, split_ratio)

In [20]:
batch_size = 100

trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                          shuffle=True, num_workers=0)

testloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                         shuffle=False, num_workers=0)

In [44]:
for epoch in range(50):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader):
        optimizer.zero_grad()
        inputs, labels = data
        predictions = net(inputs)
        _,pred = torch.max(predictions, dim=1)
        # print(pred[:10], labels[:10])
        loss = criterion(predictions, labels.long())
        # loss.requires_grad = True
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    # if i % 5 == 4:    # print every 2000 mini-batches
    print(f'Epoch {epoch}: {running_loss/len(train_set):.3f}')
    running_loss = 0.0

Epoch 0: 0.022
Epoch 1: 0.020
Epoch 2: 0.019
Epoch 3: 0.019
Epoch 4: 0.018
Epoch 5: 0.018
Epoch 6: 0.017
Epoch 7: 0.017
Epoch 8: 0.017
Epoch 9: 0.017
Epoch 10: 0.017
Epoch 11: 0.017
Epoch 12: 0.017
Epoch 13: 0.017
Epoch 14: 0.017
Epoch 15: 0.017
Epoch 16: 0.017
Epoch 17: 0.017
Epoch 18: 0.017
Epoch 19: 0.017
Epoch 20: 0.017
Epoch 21: 0.017
Epoch 22: 0.017
Epoch 23: 0.017
Epoch 24: 0.017
Epoch 25: 0.017
Epoch 26: 0.017
Epoch 27: 0.017
Epoch 28: 0.017
Epoch 29: 0.017
Epoch 30: 0.017
Epoch 31: 0.017
Epoch 32: 0.017
Epoch 33: 0.017
Epoch 34: 0.017
Epoch 35: 0.017
Epoch 36: 0.017
Epoch 37: 0.017
Epoch 38: 0.017
Epoch 39: 0.017
Epoch 40: 0.017
Epoch 41: 0.017
Epoch 42: 0.017
Epoch 43: 0.017
Epoch 44: 0.017
Epoch 45: 0.017
Epoch 46: 0.017
Epoch 47: 0.017
Epoch 48: 0.017
Epoch 49: 0.017


In [45]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct // total} %')

Accuracy: 84 %
